## Theoretical Challenges for Ocean Dynamics: ice-ocean interactions notebook

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from datetime import datetime, timedelta
import pandas
import csv
from scipy.optimize import fsolve

### 1) Data from the Ronne ice-shelf cavity (site 5)

credit: Keith Nicholls, British Antarctic Survey, UK

#### 1.1) Loading the data from a .mat file array

In [13]:
Depths5 = loadmat('Depths5c.mat') # loading the depths file
Depths5["Depths5c"] # we plot the depths (ice surface used as reference)

array([[ 679.56261513,  698.17693191,  744.71272388,  815.03347618,
         916.37808978, 1029.09811921]])

In [17]:
istart, iend = 12, 9135
AllData = loadmat('s5cInterpAll.mat')
datetimevec = np.array([datetime(year=1, month=1, day=1) + timedelta(days=time-365) for time in AllData['t'][0][istart:iend]])
U1 = np.squeeze(np.array(AllData["s5c"][0]["UV"][0]))[istart:iend]
V1 = np.squeeze(np.array(AllData["s5c"][0]["UV"][1]))[istart:iend]
T1 = np.squeeze(np.array(AllData["s5c"][0]["T"][0]))[istart:iend]
S1 = np.squeeze(np.array(AllData["s5c"][0]["S"][0]))[istart:iend]
U2 = np.squeeze(np.array(AllData["s5c"][0]["UV"][4]))[istart:iend]
V2 = np.squeeze(np.array(AllData["s5c"][0]["UV"][5]))[istart:iend]
T2 = np.squeeze(np.array(AllData["s5c"][0]["T"][4]))[istart:iend]
S2 = np.squeeze(np.array(AllData["s5c"][0]["S"][4]))[istart:iend]
melt = np.squeeze(np.array(AllData["s5c"][0]["melt"][0]))[istart:iend] # this is the melt rate directly inferred from the ApRES data

#### 1.2) Running a moving average to remove short temporal variability

In [19]:
# 24h moving average operator on hourly data -- no data loss
window_size = 24
weights = np.ones(window_size) / window_size
U1ma = np.convolve(U1, weights, mode='valid')
V1ma = np.convolve(V1, weights, mode='valid')
T1ma = np.convolve(T1, weights, mode='valid')
S1ma = np.convolve(S1, weights, mode='valid')
U2ma = np.convolve(U2, weights, mode='valid')
V2ma = np.convolve(V2, weights, mode='valid')
T2ma = np.convolve(T2, weights, mode='valid')
S2ma = np.convolve(S2, weights, mode='valid')
meltma = np.convolve(melt, weights, mode='valid')
datetimevecma = np.array([datetime(year=1, month=1, day=1) + timedelta(days=time-365) for time in np.convolve(AllData['t'][0][istart:iend], weights, mode='valid')]) 
print(datetimevecma.shape,datetimevec.shape)
print(U1ma.shape,T1ma.shape)

(9100,) (9123,)
(9100,) (9100,)


### 2) Questions
Answer the questions using the daily averaged data.

#### 2.1) Show the time history of the measured melt rate (m/y) and compute the mean and standard deviation.

#### 2.2) Compute the temporal power spectrum of the melt rate. Does it suggest that the 3-equation model should work? 

#### 2.3) Show the time history of the temperature and salinity at the two depths (T1,S1 and T2,S2); compute the mean and standard deviation and comment.

#### 2.4) Show the daily-varying melt rate as a function of the T1 temperature. 
Compute the best-fit pre-factor assuming a linear correlation. What are the pre-factor units?

#### 2.5) Show the daily-varying melt rate as a function of |U1| with |U1| the norm of the horizontal velocity field. 
Compute the best-fit pre-factor assuming a linear correlation. What are the pre-factor units?

#### 2.6) Recall the three boundary conditions that we have at an ice-ocean interface, which are based on the liquidus equation, energy conservation and salt conservation, and how they are parameterized. Considering the parameter values recommended in Jenkins et al (2010) "Observation and Parameterization of Ablation at the Base of Ronne Ice Shelf, Antarctica" (provided below), write a function that can solve the three-equation model for the unknowns  m (your prediction of the melt rate), Tb (basal temperature), and Sb (basal salinity).

In [5]:
Lam1, Lam2, Lam3 = -0.0573, 0.0832, -7.53e-8
ri, rw = 916, 1030
Lf, cw = 3.34e5, 3.974e3
Pb = 916*9.81*820 # in Pa (820 ice thickness)
GAT, GAS, CD = 0.011, 3.1e-4, 0.0097

In [6]:
TOBEDETERMINED = np.nan # just a placeholder -- remove/update TOBEDETERMINED with whatever is appropriate everywhere in this cell  
def threeeq(outputvar,inputvar): # the three equation model
  m, Tb, Sb = outputvar[0], outputvar[1], outputvar[2] 
  U, T, S = inputvar[0], inputvar[1], inputvar[2]
  eq1 = TOBEDETERMINED
  eq2 = TOBEDETERMINED
  eq3 = TOBEDETERMINED
  res = [eq1, eq2, eq3]
  return res
inputvar = [TOBEDETERMINED, TOBEDETERMINED, TOBEDETERMINED] 
outputvar = [TOBEDETERMINED, TOBEDETERMINED, TOBEDETERMINED] # initial guess
fsolve(threeeq, outputvar, inputvar)

/tmp/ipykernel_73639/262947232.py:12: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
  fsolve(threeeq, outputvar, inputvar)


array([nan, nan, nan])

#### 2.7) Plot the results for m (superimposed with the actual melt rate and the predicted melt rate "predmelt" in the dataset), Tb, and Sb and comment.

#### 2.8) Is there a set of transport coefficient values that would yield a better agreement between the measured and predicted melt rates? Can you identify it?